# Reslice and mask

Use SPM reslice function to resample the masks to each functional space, and then use fsl's ApplyMask to extract the data of interest

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

201211-11:58:37,850 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
msks = pe.Workflow(name="msks") # workflow to run the analysis

# Grab and save data

In [3]:
# THIS CODE DEFINITELY WORKS
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['masks','con', 'CNR', 'RMS']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'masks': [['sub']],
                           'con':[['sub', 'TR']],
                           'CNR': [['sub', 'TR']],
                           'RMS': [['sub', 'TR']]}
dg.inputs.field_template = {'masks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-%s/*/*.nii.gz',
                            'con': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/con_0001.nii.gz',
                            'CNR': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/CNR_*.nii.gz',
                            'RMS': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/RMS_CNR_*.nii.gz'}

In [4]:
# # debugging
# dg.inputs.sub = '05'
# dg.inputs.TR = '1920'
# res = dg.run()
# res.outputs

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_gunzipmask([0-9]*)', ''),
                 ('_mask-CNR([0-9]*)', ''),
                 ('_mask-RMS([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

In [7]:
# THIS WORKS
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['05']), ('TR', ['1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

# Gunzip node

In [8]:
gunzipmany = pe.MapNode(Gunzip(), name='gunzipmask', iterfield=['in_file'])

In [9]:
gunzipcon = pe.Node(Gunzip(), name='gunzipcon')

# Reslice node

In [10]:
reslice = pe.MapNode(spm.Reslice(), iterfield=['in_file'], name='reslice')

# Gzip interface 
Required to get data from .nii (spm) to .nii.gz (fsl)

In [11]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')
        
# if __name__ == '__main__':

#     zipper = GZipTask(input_file='an_existing_file')
#     print zipper.cmdline 
#     zipper.run()

Functions to concatenate the filelists into complementary lists:

In [12]:
def cnrCat(fullFileList, nMasks=9):
    return fullFileList*nMasks

In [13]:
#cnrCat(res.outputs.CNR)

In [14]:
def MskCat(fullFileList, nCon=7):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

In [15]:
#MskCat(res.outputs.masks)

# define out_file names for the fsl masking functions

In [16]:
# first get a list of all the confounds files for the single echo TRs to which I will apply the printCensored and printMotion functions below
CNR_names = ['CNR_0001', 'CNR_0002', 'CNR_0003', 'CNR_0005', 'CNR_0006', 'CNR_0007', 'CNR_0009']
RMS_names = ['RMS_CNR_0001', 'RMS_CNR_0002', 'RMS_CNR_0003', 'RMS_CNR_0005', 'RMS_CNR_0006', 'RMS_CNR_0007', 'RMS_CNR_0009']
masks = ['FEF', 'IPS', 'LOC', 'CN', 'GPe', 'GPi', 'Put', 'STN', 'VS']
CNR_fnames = [i+'_'+n+'.nii.gz' for i in masks for n in CNR_names]
RMS_fnames = [i+'_'+n+'.nii.gz' for i in masks for n in RMS_names]
CNR_fnames

['FEF_CNR_0001.nii.gz',
 'FEF_CNR_0002.nii.gz',
 'FEF_CNR_0003.nii.gz',
 'FEF_CNR_0005.nii.gz',
 'FEF_CNR_0006.nii.gz',
 'FEF_CNR_0007.nii.gz',
 'FEF_CNR_0009.nii.gz',
 'IPS_CNR_0001.nii.gz',
 'IPS_CNR_0002.nii.gz',
 'IPS_CNR_0003.nii.gz',
 'IPS_CNR_0005.nii.gz',
 'IPS_CNR_0006.nii.gz',
 'IPS_CNR_0007.nii.gz',
 'IPS_CNR_0009.nii.gz',
 'LOC_CNR_0001.nii.gz',
 'LOC_CNR_0002.nii.gz',
 'LOC_CNR_0003.nii.gz',
 'LOC_CNR_0005.nii.gz',
 'LOC_CNR_0006.nii.gz',
 'LOC_CNR_0007.nii.gz',
 'LOC_CNR_0009.nii.gz',
 'CN_CNR_0001.nii.gz',
 'CN_CNR_0002.nii.gz',
 'CN_CNR_0003.nii.gz',
 'CN_CNR_0005.nii.gz',
 'CN_CNR_0006.nii.gz',
 'CN_CNR_0007.nii.gz',
 'CN_CNR_0009.nii.gz',
 'GPe_CNR_0001.nii.gz',
 'GPe_CNR_0002.nii.gz',
 'GPe_CNR_0003.nii.gz',
 'GPe_CNR_0005.nii.gz',
 'GPe_CNR_0006.nii.gz',
 'GPe_CNR_0007.nii.gz',
 'GPe_CNR_0009.nii.gz',
 'GPi_CNR_0001.nii.gz',
 'GPi_CNR_0002.nii.gz',
 'GPi_CNR_0003.nii.gz',
 'GPi_CNR_0005.nii.gz',
 'GPi_CNR_0006.nii.gz',
 'GPi_CNR_0007.nii.gz',
 'GPi_CNR_0009.nii.gz',

# Use FSL to mask the CNR and the RMS images

In [17]:
maskCNR = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-CNR')
maskCNR.inputs.out_file = CNR_fnames

In [18]:
maskRMS = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-RMS')
maskRMS.inputs.out_file = RMS_fnames

In [19]:
msks.connect([(datinf, dg, [('sub', 'sub')]),
              (datinf, dg, [('TR', 'TR')]),
              (dg, ds, [(('con', printSubPath), 'container')]),
              (dg, gunzipmany, [('masks', 'in_file')]),
              (dg, gunzipcon, [('con', 'in_file')]),
              (gunzipmany, reslice, [('out_file', 'in_file')]),
              (gunzipcon, reslice, [('out_file', 'space_defining')]),
              (reslice, gzip, [('out_file', 'in_file')]),
              (gzip, ds, [('out_file', 'MASKS.@RESLICE')]),
              (gzip, maskCNR, [(('out_file', MskCat), 'mask_file')]),
              (dg, maskCNR, [(('CNR', cnrCat), 'in_file')]),
              (maskCNR, ds, [('out_file', 'MASKS.CNR')]),
              (gzip, maskRMS, [(('out_file', MskCat), 'mask_file')]),
              (dg, maskRMS, [(('RMS', cnrCat), 'in_file')]),
              (maskRMS, ds, [('out_file', 'MASKS.RMS')])
])
res = msks.run()

# Reslice node

201211-11:58:40,417 nipype.workflow INFO:
	 Workflow msks settings: ['check', 'execution', 'logging', 'monitoring']
201211-11:58:40,534 nipype.workflow INFO:
	 Running serially.
201211-11:58:40,535 nipype.workflow INFO:
	 [Node] Setting-up "msks.data-grabber" in "/tmp/tmphgo2j_4_/msks/_TR_1920_sub_05/data-grabber".
201211-11:58:40,541 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
201211-11:58:40,572 nipype.workflow INFO:
	 [Node] Finished "msks.data-grabber".
201211-11:58:40,574 nipype.workflow INFO:
	 [Node] Setting-up "msks.gunzipcon" in "/tmp/tmp24u7hlal/msks/_TR_1920_sub_05/gunzipcon".
201211-11:58:40,579 nipype.workflow INFO:
	 [Node] Running "gunzipcon" ("nipype.algorithms.misc.Gunzip")
201211-11:58:40,627 nipype.workflow INFO:
	 [Node] Finished "msks.gunzipcon".
201211-11:58:40,628 nipype.workflow INFO:
	 [Node] Setting-up "msks.gunzipmask" in "/tmp/tmpex4wkzrh/msks/_TR_1920_sub_05/gunzipmask".
201211-11:58:40,634 nipype.workflow INFO

201211-11:59:18,203 nipype.workflow INFO:
	 [Node] Setting-up "_gzip2" in "/tmp/tmpl1_du7ou/msks/_TR_1920_sub_05/gzip/mapflow/_gzip2".
201211-11:59:18,208 nipype.workflow INFO:
	 [Node] Running "_gzip2" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpex4wkzrh/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask2/rLOC_trans.nii
201211-11:59:18,369 nipype.workflow INFO:
	 [Node] Finished "_gzip2".
201211-11:59:18,372 nipype.workflow INFO:
	 [Node] Setting-up "_gzip3" in "/tmp/tmpl1_du7ou/msks/_TR_1920_sub_05/gzip/mapflow/_gzip3".
201211-11:59:18,376 nipype.workflow INFO:
	 [Node] Running "_gzip3" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpex4wkzrh/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask3/rCN_trans.nii
201211-11:59:18,517 nipype.workflow INFO:
	 [Node] Finished "_gzip3".
201211-11:59:18,518 nipype.workflow INFO:
	 [Node] Setting-up "_gzip4" in "/tmp/tmpl1_du7ou/msks/_TR_1920_sub_05/gzip/mapflow/_gzip4".
201211-11:59:18,52

201211-11:59:21,417 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS8".
201211-11:59:21,419 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS9" in "/tmp/tmpsf_ue9kh/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS9".
201211-11:59:21,424 nipype.workflow INFO:
	 [Node] Running "_mask-RMS9" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/RMS_CNR_0003.nii.gz -mas /tmp/tmpex4wkzrh/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask1/rIPS_trans.nii.gz IPS_RMS_CNR_0003.nii.gz
201211-11:59:21,683 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS9".
201211-11:59:21,686 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS10" in "/tmp/tmpsf_ue9kh/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS10".
201211-11:59:21,691 nipype.workflow INFO:
	 [Node] Running "_mask-RMS10" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP

201211-11:59:24,489 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS22".
201211-11:59:24,493 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS23" in "/tmp/tmpsf_ue9kh/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS23".
201211-11:59:24,498 nipype.workflow INFO:
	 [Node] Running "_mask-RMS23" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/RMS_CNR_0003.nii.gz -mas /tmp/tmpex4wkzrh/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask3/rCN_trans.nii.gz CN_RMS_CNR_0003.nii.gz
201211-11:59:24,731 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS23".
201211-11:59:24,733 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS24" in "/tmp/tmpsf_ue9kh/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS24".
201211-11:59:24,737 nipype.workflow INFO:
	 [Node] Running "_mask-RMS24" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STR

201211-11:59:27,618 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS36".
201211-11:59:27,620 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS37" in "/tmp/tmpsf_ue9kh/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS37".
201211-11:59:27,625 nipype.workflow INFO:
	 [Node] Running "_mask-RMS37" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/RMS_CNR_0003.nii.gz -mas /tmp/tmpex4wkzrh/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask5/rGPi_trans.nii.gz GPi_RMS_CNR_0003.nii.gz
201211-11:59:27,832 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS37".
201211-11:59:27,834 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS38" in "/tmp/tmpsf_ue9kh/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS38".
201211-11:59:27,839 nipype.workflow INFO:
	 [Node] Running "_mask-RMS38" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/S

...

201211-12:15:40,607 nipype.workflow INFO:
	 [Node] Finished "_mask-CNR60".
201211-12:15:40,609 nipype.workflow INFO:
	 [Node] Setting-up "_mask-CNR61" in "/tmp/tmp_s0s8xp8/msks/_TR_700_sub_01/mask-CNR/mapflow/_mask-CNR61".
201211-12:15:40,614 nipype.workflow INFO:
	 [Node] Running "_mask-CNR61" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/CNR_0007.nii.gz -mas /tmp/tmphny3r7ut/msks/_TR_700_sub_01/gunzipmask/mapflow/_gunzipmask8/rVS_trans.nii.gz VS_CNR_0007.nii.gz
201211-12:15:40,806 nipype.workflow INFO:
	 [Node] Finished "_mask-CNR61".
201211-12:15:40,808 nipype.workflow INFO:
	 [Node] Setting-up "_mask-CNR62" in "/tmp/tmp_s0s8xp8/msks/_TR_700_sub_01/mask-CNR/mapflow/_mask-CNR62".
201211-12:15:40,813 nipype.workflow INFO:
	 [Node] Running "_mask-CNR62" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivat

201211-12:15:41,294 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR18/LOC_CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///LOC_CNR_0006.nii.gz
201211-12:15:41,296 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR19/LOC_CNR_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///LOC_CNR_0007.nii.gz
201211-12:15:41,318 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR20/LOC_CNR_0009.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///LOC_CNR_0009.nii.gz
201211-12:15:41,327 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR21/CN_CNR_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01

201211-12:15:41,596 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR50/STN_CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///STN_CNR_0002.nii.gz
201211-12:15:41,606 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR51/STN_CNR_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///STN_CNR_0003.nii.gz
201211-12:15:41,615 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR52/STN_CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///STN_CNR_0005.nii.gz
201211-12:15:41,618 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR53/STN_CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-0

201211-12:15:41,744 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS19/LOC_RMS_CNR_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///LOC_RMS_CNR_0007.nii.gz
201211-12:15:41,746 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS20/LOC_RMS_CNR_0009.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///LOC_RMS_CNR_0009.nii.gz
201211-12:15:41,747 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS21/CN_RMS_CNR_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///CN_RMS_CNR_0001.nii.gz
201211-12:15:41,750 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS22/CN_RMS_CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRI

201211-12:15:41,875 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS50/STN_RMS_CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///STN_RMS_CNR_0002.nii.gz
201211-12:15:41,876 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS51/STN_RMS_CNR_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///STN_RMS_CNR_0003.nii.gz
201211-12:15:41,878 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS52/STN_RMS_CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///STN_RMS_CNR_0005.nii.gz
201211-12:15:41,879 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS53/STN_RMS_CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/S